In [1]:
import pandas as pd
from google.colab import files
import numpy as np
pd.set_option('display.max_columns', None)

In [16]:
# --- Cell 2 (Corrected) ---
import pandas as pd
from google.colab import files
import numpy as np

pd.set_option('display.max_columns', None)

try:
    # --- THIS IS THE FIX ---
    # We've added on_bad_lines='warn'
    # This will print a warning for every row it's forced to skip.
    print("Attempting to load 'tyre_degradation.csv' (Warning Mode)...")
    df_tyre = pd.read_csv('tyre_degradation.csv', on_bad_lines='warn')

    # We'll load the other file normally
    df_winner = pd.read_csv('race_winner_prediction.csv')

    print("\n--- 1. Tyre Degradation Data (Raw) ---")
    df_tyre.info()
    print("\n")

    print("--- 2. Race Winner Data (Raw) ---")
    df_winner.info()

except FileNotFoundError:
    print("ERROR: CSV files not found.")
    print("Please upload 'tyre_degradation.csv' and 'race_winner_prediction.csv' to this Colab session.")

Attempting to load 'tyre_degradation.csv' (Warning Mode)...

--- 1. Tyre Degradation Data (Raw) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196706 entries, 0 to 196705
Data columns (total 25 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   country_name       196706 non-null  object 
 1   session_name       196706 non-null  object 
 2   date_start         196706 non-null  object 
 3   driver_number      196706 non-null  int64  
 4   lap_number         196706 non-null  int64  
 5   duration_sector_1  171134 non-null  float64
 6   duration_sector_2  194889 non-null  float64
 7   duration_sector_3  191584 non-null  float64
 8   i1_speed           179865 non-null  float64
 9   i2_speed           194901 non-null  float64
 10  st_speed           181765 non-null  float64
 11  is_pit_out_lap     196706 non-null  bool   
 12  lap_duration       189373 non-null  float64
 13  segments_sector_1  194353 non-null  object 
 14  

In [17]:
print("--- Processing Tyre Degradation Data ---")

# This dictionary will hold our stats
imputation_report = {
    'total_laps_started': len(df_tyre),
    'laps_dropped_out_laps': 0,
    'laps_dropped_missing_target': 0,
    'total_laps_kept': 0,
    'cells_imputed': 0,
    'total_cells_checked': 0,
    'percent_of_data_imputed': 0.0
}

# Step 1: Create a copy
df_tyre_processed = df_tyre.copy()

# --- Step 2: (THE CONCRETE FIX) Impute and Filter Laps ---
# Your file has no 'pit_duration'. The 'is_pit_out_lap' column likely
# has NaN values for new races. We fix this.

# 1. Fill all NaN values in 'is_pit_out_lap' with False.
# This assumes that if a lap is not explicitly marked as an out-lap, it is a valid lap.
original_nans = df_tyre_processed['is_pit_out_lap'].isna().sum()
df_tyre_processed['is_pit_out_lap'].fillna(False, inplace=True)
print(f"Step 1: Filled {original_nans} missing 'is_pit_out_lap' values with False.")

# 2. NOW, we can safely filter. We only drop laps that were EXPLICITLY 'True'.
original_rows = len(df_tyre_processed)
df_tyre_processed = df_tyre_processed[
    df_tyre_processed['is_pit_out_lap'] == False
].copy()
rows_after_filter = len(df_tyre_processed)
imputation_report['laps_dropped_out_laps'] = original_rows - rows_after_filter
print(f"Step 2: Removed {imputation_report['laps_dropped_out_laps']} confirmed out-laps.")

# --- Step 3: Handle missing TARGET variable ---
# We will ONLY drop a row if its 'lap_duration' is missing.
original_rows = len(df_tyre_processed)
df_tyre_processed.dropna(
    subset=['lap_duration'],
    inplace=True
)
rows_after_dropna = len(df_tyre_processed)
imputation_report['laps_dropped_missing_target'] = original_rows - rows_after_dropna
imputation_report['total_laps_kept'] = rows_after_dropna
print(f"Step 3: Removed {imputation_report['laps_dropped_missing_target']} laps with missing 'lap_duration' (our target).")
print(f"   -> Total 'usable' laps kept: {rows_after_dropna}") # This number should now be much higher

# Step 4: Drop complex/unused columns
cols_to_drop = [col for col in df_tyre_processed.columns if 'segments_' in col]
df_tyre_processed.drop(columns=cols_to_drop, inplace=True)
print(f"Step 4: Dropped complex 'segments' columns.")

# --- Step 5: Smart Grouped Imputation (Your Idea) ---
print("Step 5: Imputing all missing features using 'Grouped Medians'...")

# Define columns to fill and their groups
perf_cols = [
    'duration_sector_1', 'duration_sector_2', 'duration_sector_3',
    'i1_speed', 'i2_speed', 'st_speed'
]
weather_cols = ['air_temperature', 'track_temperature', 'rainfall', 'wind_speed']

total_cells_to_check = len(df_tyre_processed) * (len(perf_cols) + len(weather_cols))
imputation_report['total_cells_checked'] = total_cells_to_check
total_filled_cells = 0

# Impute Performance Features (grouped by driver-session)
for col in perf_cols:
    if col in df_tyre_processed.columns:
        missing_count = df_tyre_processed[col].isna().sum()
        total_filled_cells += missing_count

        grouped_median = df_tyre_processed.groupby(['session_key', 'driver_number'])[col].transform('median')
        df_tyre_processed[col].fillna(grouped_median, inplace=True)

        global_median = df_tyre_processed[col].median()
        df_tyre_processed[col].fillna(global_median, inplace=True)

        if missing_count > 0:
            print(f"  -> Filled {missing_count} missing '{col}' values using driver/session medians.")

# Impute Weather Features (grouped by session)
for col in weather_cols:
    if col in df_tyre_processed.columns:
        missing_count = df_tyre_processed[col].isna().sum()
        total_filled_cells += missing_count

        session_median = df_tyre_processed.groupby('session_key')[col].transform('median')
        df_tyre_processed[col].fillna(session_median, inplace=True)

        global_median = df_tyre_processed[col].median()
        df_tyre_processed[col].fillna(global_median, inplace=True)

        if missing_count > 0:
            print(f"  -> Filled {missing_count} missing '{col}' values using session medians.")

# Fill 'compound' & 'tyre_age'
df_tyre_processed['compound'].fillna('UNKNOWN', inplace=True)
df_tyre_processed['tyre_age'].fillna(0, inplace=True)
print("  -> Filled missing 'compound' with 'UNKNOWN' and 'tyre_age' with 0.")

# Step 6: Sort the Data
df_tyre_processed.sort_values(
    by=['session_key', 'driver_number', 'lap_number'],
    inplace=True
)
print("Step 6: Sorted data by session, driver, and lap.")

# Step 7: Final Report
imputation_report['cells_imputed'] = total_filled_cells
if total_cells_to_check > 0:
    imputation_report['percent_of_data_imputed'] = (total_filled_cells / total_cells_to_check) * 100

print("\n--- Final Tyre Data Info ---")
df_tyre_processed.info()

print("\n--- PREPROCESSING REPORT (TYRE DEG) ---")
print(f"Total Laps in Original File: {imputation_report['total_laps_started']}")
print(f"Laps Kept (Final):           {imputation_report['total_laps_kept']}")
print(f"Laps Dropped (Total):        {imputation_report['laps_dropped_out_laps'] + imputation_report['laps_dropped_missing_target']}")
print(f"----------------------------------------")
print(f"Cells Checked for Imputation:  {imputation_report['total_cells_checked']}")
print(f"Cells Imputed (Filled):      {imputation_report['cells_imputed']}")
print(f"IMPUTATION RATE:             {imputation_report['percent_of_data_imputed']:.2f}%")
print("----------------------------------------")

--- Processing Tyre Degradation Data ---
Step 1: Filled 0 missing 'is_pit_out_lap' values with False.
Step 2: Removed 37745 confirmed out-laps.
Step 3: Removed 2767 laps with missing 'lap_duration' (our target).
   -> Total 'usable' laps kept: 156194
Step 4: Dropped complex 'segments' columns.
Step 5: Imputing all missing features using 'Grouped Medians'...
  -> Filled 362 missing 'duration_sector_1' values using driver/session medians.
  -> Filled 264 missing 'duration_sector_2' values using driver/session medians.
  -> Filled 1581 missing 'duration_sector_3' values using driver/session medians.
  -> Filled 16029 missing 'i1_speed' values using driver/session medians.
  -> Filled 253 missing 'i2_speed' values using driver/session medians.


/tmp/ipython-input-774557234.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_tyre_processed['is_pit_out_lap'].fillna(False, inplace=True)
/tmp/ipython-input-774557234.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace

  -> Filled 8277 missing 'st_speed' values using driver/session medians.
  -> Filled missing 'compound' with 'UNKNOWN' and 'tyre_age' with 0.
Step 6: Sorted data by session, driver, and lap.

--- Final Tyre Data Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 156194 entries, 2311 to 156561
Data columns (total 22 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   country_name       156194 non-null  object 
 1   session_name       156194 non-null  object 
 2   date_start         156194 non-null  object 
 3   driver_number      156194 non-null  int64  
 4   lap_number         156194 non-null  int64  
 5   duration_sector_1  156194 non-null  float64
 6   duration_sector_2  156194 non-null  float64
 7   duration_sector_3  156194 non-null  float64
 8   i1_speed           156194 non-null  float64
 9   i2_speed           156194 non-null  float64
 10  st_speed           156194 non-null  float64
 11  is_pit_out_lap     156194 no

/tmp/ipython-input-774557234.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_tyre_processed[col].fillna(session_median, inplace=True)
/tmp/ipython-input-774557234.py:93: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [18]:
# Save the processed data to a new CSV
df_tyre_processed.to_csv('new_tyre_degradation_processed.csv', index=False)

# Trigger the download
print("\nDownloading 'tyre_degradation_processed.csv'...")
files.download('tyre_degradation_processed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
# --- This is a new diagnostic cell. Run it after loading your data. ---

print("--- Analyzing Raw Tyre Data (190k rows) ---")

if 'df_tyre' in locals():
    # 1. Load the raw data
    total_rows = len(df_tyre)
    print(f"Total Rows in Original File: {total_rows}")
    print("-" * 40)

    # 2. Count the laps we are filtering
    out_laps = len(df_tyre[df_tyre['is_pit_out_lap'] == True])
    in_laps = len(df_tyre[df_tyre['pit_duration'].notna()])

    # 3. Find laps that are NOT in or out laps
    performance_laps_df = df_tyre[
        (df_tyre['is_pit_out_lap'] == False) &
        (df_tyre['pit_duration'].isna())
    ]
    total_performance_laps = len(performance_laps_df)

    # 4. From those "real" laps, find how many are missing our target
    missing_target = performance_laps_df['lap_duration'].isna().sum()

    # 5. Calculate the final dataset
    final_rows = total_performance_laps - missing_target
    total_rows_dropped = total_rows - final_rows

    print(f"Laps Identified as OUT-LAPS:     {out_laps}")
    print(f"Laps Identified as IN-LAPS:      {in_laps}")
    print(f"--------------------------------------------------")
    print(f"Total 'Real' Performance Laps: {total_performance_laps}")
    print(f"Laps Missing Target ('lap_duration'): {missing_target}")
    print(f"--------------------------------------------------")
    print(f"Total Laps We Will KEEP:       {final_rows}")
    print(f"Total Laps We Will DROP:       {total_rows_dropped}")

    # Check if the numbers add up
    # Note: Some in-laps might also be out-laps (e.g., pit stop penalty), so we can't just add them.
    print(f"\nFinal row count (approx. 100k) matches: {final_rows == 100000}")


else:
    print("Please run Cell 2 to load 'df_tyre' before running this diagnostic.")

--- Analyzing Raw Tyre Data (190k rows) ---
Total Rows in Original File: 196706
----------------------------------------
Laps Identified as OUT-LAPS:     37745
Laps Identified as IN-LAPS:      28085
--------------------------------------------------
Total 'Real' Performance Laps: 156780
Laps Missing Target ('lap_duration'): 2767
--------------------------------------------------
Total Laps We Will KEEP:       154013
Total Laps We Will DROP:       42693

Final row count (approx. 100k) matches: False
